In [39]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim

import dlc_practical_prologue as prologue

if torch.cuda.is_available():  
  dev="cuda:0"
  device = torch.device(dev) 

In [40]:
pairs = prologue.generate_pair_sets(1000)

train_input=pairs[0].cuda()
train_target=pairs[1].cuda()
train_classes=pairs[2].cuda()
test_input=pairs[3].cuda()
test_target=pairs[4].cuda()
test_classes=pairs[5].cuda()

train_classes1=prologue.convert_to_one_hot_labels(train_input,train_classes[:,0])
train_classes2=prologue.convert_to_one_hot_labels(train_input,train_classes[:,1])
train_classes_hot=torch.cat((train_classes1,train_classes2),1)

train_classes1=prologue.convert_to_one_hot_labels(train_input,train_classes[:,0])
train_classes2=prologue.convert_to_one_hot_labels(train_input,train_classes[:,1])
test_classes_hot=torch.cat((train_classes1,train_classes2),1)

In [41]:
train_classes_hot=torch.cat((train_classes1,train_classes2),1)
train_classes_hot[1], train_target[1]


(tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0.], device='cuda:0'),
 tensor(0, device='cuda:0'))

In [32]:

class MLPAux(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(392, 500)
        self.fc2 = nn.Linear(500, 20)
        self.fc3 = nn.Linear(20, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        y = self.fc2(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x, y

In [21]:
def train_model(model, train_input, train_target, train_classes):
    criterion  = nn.CrossEntropyLoss()
    criterion2 = nn.BCEWithLogitsLoss()
    optimizer  = optim.SGD(model.parameters(), lr = 1e-3)
    nb_epochs  = 100
    mini_batch_size= 100

    for e in range(nb_epochs):
        for b in range(0, train_input.size(0), mini_batch_size):
            output, output2 = model(train_input.narrow(0, b, mini_batch_size).view((mini_batch_size,-1)))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size)) + criterion2(output2, train_classes.narrow(0, b, mini_batch_size))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

In [37]:
def compute_nb_errors(model, data_input, data_target):

    nb_data_errors = 0
    mini_batch_size= 100
    for b in range(0, data_input.size(0), mini_batch_size):
        output, output2 = model(data_input.narrow(0, b, mini_batch_size).view((mini_batch_size,-1)))
        _, predicted_classes = torch.max(output, 1)
        for k in range(mini_batch_size):
            if data_target[b + k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1

    return nb_data_errors

In [43]:
train_E=[]
test_E=[]

for k in range(5):
    model=MLPAux().cuda()
    train_model(model,train_input,train_target,train_classes_hot)
    train_E.append(compute_nb_errors(model, train_input, train_target) / train_input.size(0) * 100)
    test_E.append(compute_nb_errors(model, test_input, test_target) / test_input.size(0) * 100)
    print('train_error {:.02f}% test_error {:.02f}%'.format(train_E[-1],test_E[-1]))

train_error 2.30% test_error 22.50%
train_error 7.00% test_error 27.40%
train_error 3.20% test_error 21.90%
train_error 1.20% test_error 21.50%
train_error 2.90% test_error 26.60%
